In [1]:
import os
from typing import List

import geopandas as gpd
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np
import pandas as pd
import seaborn as sns
from metr.dataset.interpolator import (Interpolator,
                                       IterativeRandomForestInterpolator,
                                       LinearInterpolator, SplineInterpolator,
                                       TimeMeanFillInterpolator)
from metr.dataset.metr_imc.converter.graph_sensor_locations import \
    SensorNetworkView
from metr.dataset.metr_imc.generator import MetrImcSubsetGenerator
from scipy import stats
from tqdm import tqdm
from metr.subset import MetrSubset

In [2]:
yn_selected_roads: gpd.GeoDataFrame = gpd.read_file(
    "../datasets/metr-imc-subsets/selected_road.shp"
)
yn_selected_roads

,index,sensor_id,geometry
0,71,1650026001,POINT (126.7067 37.39807)
1,73,1650042800,POINT (126.73509 37.42093)
2,127,1650074500,POINT (126.71392 37.40455)
3,129,1650232500,POINT (126.73677 37.40341)
4,131,1650388801,POINT (126.69317 37.40302)
...,...,...,...
556,16293,1650037302,POINT (126.72667 37.41513)
557,16301,1650002302,POINT (126.71535 37.39645)
558,16311,1650231700,POINT (126.73463 37.40239)
559,16319,1640011900,POINT (126.67742 37.40453)


In [11]:
yn_selected_roads.dtypes

index           int64
sensor_id      object
geometry     geometry
dtype: object

In [3]:
metr_subset = MetrSubset("../datasets/metr-imc/")
metr_subset

2024/09/20 14:17:25 metr.components.metr_imc.traffic_data [INFO] Data sorted by time: h


In [4]:
metr_subset.metr_imc.data
traffic_data = metr_subset.metr_imc
traffic_data.data

,1660041701,1610025902,1680058906,1660001801,1660022100,1640014200,1670010800,1610006303,1610005701,1630023200,...,1610002000,1610013813,1663186901,1630047901,1670012500,1650001102,1670153700,1640009500,1650225300,1660030901
2023-09-01 00:00:00,0.0,36.0,98.0,124.0,NaN,33.0,6.0,279.0,325.0,80.0,...,NaN,0.0,99.0,0.0,NaN,63.0,19.0,NaN,0.0,0.0
2023-09-01 01:00:00,0.0,13.0,39.0,74.0,NaN,6.0,0.0,240.0,285.0,37.0,...,NaN,0.0,40.0,0.0,NaN,15.0,0.0,NaN,0.0,0.0
2023-09-01 02:00:00,0.0,18.0,37.0,44.0,NaN,5.0,0.0,235.0,226.0,16.0,...,NaN,0.0,33.0,0.0,NaN,14.0,0.0,NaN,0.0,0.0
2023-09-01 03:00:00,0.0,0.0,21.0,47.0,NaN,0.0,0.0,319.0,150.0,17.0,...,NaN,0.0,13.0,0.0,NaN,41.0,4.0,NaN,0.0,0.0
2023-09-01 04:00:00,0.0,6.0,79.0,141.0,NaN,6.0,0.0,472.0,208.0,53.0,...,NaN,0.0,36.0,0.0,NaN,57.0,15.0,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-31 19:00:00,375.0,321.0,268.0,412.0,109.0,365.0,32.0,841.0,805.0,1081.0,...,151.0,157.0,359.0,0.0,NaN,277.0,189.0,10.0,23.0,50.0
2024-08-31 20:00:00,356.0,203.0,208.0,403.0,90.0,296.0,8.0,597.0,923.0,870.0,...,110.0,122.0,348.0,6.0,NaN,204.0,177.0,3.0,10.0,53.0
2024-08-31 21:00:00,289.0,195.0,170.0,333.0,93.0,216.0,13.0,522.0,674.0,808.0,...,79.0,72.0,338.0,0.0,NaN,175.0,96.0,0.0,0.0,22.0
2024-08-31 22:00:00,212.0,116.0,108.0,312.0,63.0,177.0,0.0,526.0,659.0,692.0,...,29.0,40.0,276.0,0.0,NaN,121.0,67.0,0.0,0.0,6.0


In [5]:
target_road_list = yn_selected_roads["sensor_id"].tolist()

In [8]:
metr_subset.sensor_filter = target_road_list
metr_subset.metr_imc.data

2024/09/20 14:17:57 metr.components.metr_imc.traffic_data [WARNING] The following sensors do not exist in the data:
1650002302, 1630174701, 1630175001, 1630174801, 1630167000, 1650002402, 1630174901, 1630175101, 1640320200
2024/09/20 14:18:00 metr.components.distance_imc [WARNING] The following sensors do not exist in the data: 1650002302, 1630174701, 1630175001, 1630174801, 1630167000, 1650002402, 1630174901, 1630175101, 1640320200
2024/09/20 14:20:28 metr.components.graph_sensor_locations [WARNING] The following sensors do not exist in the data: 1650391101, 1640052400, 1650024702, 1650353001, 1640020002, 1640014500, 1640014800, 1650000400, 1650001203, 1650374101, 1650351901, 1650072600, 1650353301, 1650231700, 1640008800, 1640325901, 1640022000, 1650015200, 1650232500, 1650381201, 1640042500, 1650000600, 1640050500, 1630174601, 1650374201, 1640046900, 1640041100, 1640326001, 1650388801, 1640025700, 1650083101, 1650091001, 1640001500, 1640016105, 1640049900, 1650031401, 1640024200, 16

,1640014200,1640033000,1650402301,1640038000,1650380301,1640019901,1650389601,1640245800,1650351901,1640001500,...,1630008000,1640244000,1650352901,1640041100,1650039400,1640016700,1640020002,1650001102,1640009500,1650225300
2023-09-01 00:00:00,33.0,0.0,0.0,0.0,10.0,0.0,35.0,NaN,0.0,26.0,...,NaN,0.0,0.0,13.0,126.0,0.0,0.0,63.0,NaN,0.0
2023-09-01 01:00:00,6.0,0.0,0.0,0.0,4.0,0.0,23.0,NaN,0.0,17.0,...,NaN,0.0,0.0,7.0,94.0,0.0,0.0,15.0,NaN,0.0
2023-09-01 02:00:00,5.0,0.0,0.0,0.0,11.0,0.0,6.0,NaN,0.0,4.0,...,NaN,0.0,0.0,0.0,118.0,0.0,0.0,14.0,NaN,0.0
2023-09-01 03:00:00,0.0,0.0,0.0,0.0,13.0,0.0,14.0,NaN,0.0,3.0,...,NaN,0.0,0.0,0.0,125.0,0.0,0.0,41.0,NaN,0.0
2023-09-01 04:00:00,6.0,0.0,0.0,0.0,23.0,0.0,12.0,NaN,0.0,16.0,...,NaN,3.0,0.0,7.0,241.0,0.0,0.0,57.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-31 19:00:00,365.0,0.0,0.0,3.0,32.0,0.0,165.0,0.0,35.0,135.0,...,278.0,121.0,48.0,185.0,607.0,13.0,NaN,277.0,10.0,23.0
2024-08-31 20:00:00,296.0,0.0,0.0,0.0,28.0,0.0,135.0,0.0,18.0,120.0,...,253.0,98.0,44.0,124.0,476.0,25.0,NaN,204.0,3.0,10.0
2024-08-31 21:00:00,216.0,0.0,0.0,0.0,5.0,0.0,114.0,0.0,3.0,124.0,...,194.0,117.0,18.0,74.0,405.0,12.0,NaN,175.0,0.0,0.0
2024-08-31 22:00:00,177.0,6.0,0.0,0.0,12.0,6.0,87.0,0.0,0.0,84.0,...,143.0,105.0,13.0,69.0,305.0,17.0,NaN,121.0,0.0,0.0


In [10]:
print(len(metr_subset.distances_imc.sensor_filter))
print(len(target_road_list))

552
561
